In [5]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math

In [6]:
stations_df = pd.read_csv("Stazioni_Meteorologiche.csv")
stations_df

,IdSensore,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,22006,Umidità Relativa,%,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
1,22007,Direzione Vento,°,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
2,22003,Temperatura,°C,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
3,22009,Velocità Vento,m/s,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
4,17612,Altezza Neve,cm,1545,San Siro Alpe Rescascìa,1285,CO,07/11/2019,NaN,N,5102069,518430,9.238323,46.071947,"(46.071947417956004, 9.238323435918229)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,10787,Velocità Vento,m/s,1266,Bertonico v.Moro,65,LO,12/04/2022,NaN,N,5009105,552297,9.666250,45.233494,"(45.23349364130374, 9.666250375296032)"
1244,19024,Radiazione Globale,W/m²,1422,Bergamo v.Maffei,249,BG,06/11/2013,NaN,N,5060404,552526,9.674653,45.695197,"(45.6951969397896, 9.674652773656335)"
1245,7048,Velocità Vento,m/s,695,Ponti sul Mincio v.San Martino,113,MN,19/12/2006,03/03/2021,S,5030184,631718,10.683357,45.412772,"(45.41277169168634, 10.683357343786835)"
1246,6990,Radiazione Globale,W/m²,679,Valmadrera v.Pozzi,247,LC,02/02/2006,NaN,N,5076576,527307,9.351658,45.842207,"(45.842207261022544, 9.351658166127958)"


In [9]:
def rearange_date_d_m_y(date_str):
    if date_str is None: return None
    date_split = date_str.split("/")
    return f"{date_split[2]}-{date_split[1]}-{date_split[0]}"

def rearange_date_m_d_y(date_str):
    if date_str is None: return None
    date_split = date_str.split("/")
    return f"{date_split[2]}-{date_split[0]}-{date_split[1]}"

def map_sensor_type(sensor_type):
    if sensor_type == "Umidità Relativa": return "Humidity"
    if sensor_type == "Direzione Vento": return "Wind Direction"
    if sensor_type == "Temperatura": return "Temperature"
    if sensor_type == "Velocità Vento": return "Wind Velocity"
    if sensor_type == "Altezza Neve": return "Snow Depth"
    if sensor_type == "Precipitazione": return "Precipitation"
    if sensor_type == "Radiazione Globale": return "Global Radiation"
    if sensor_type == "Livello Idrometrico": return "Hidrometric Level"
    return None

0               Humidity
1         Wind Direction
2            Temperature
3          Wind Velocity
4             Snow Depth
              ...       
1243       Wind Velocity
1244    Global Radiation
1245       Wind Velocity
1246    Global Radiation
1247         Temperature
Name: Tipologia, Length: 1248, dtype: object

In [11]:
stations_df = stations_df.drop('location', axis=1)
stations_df = stations_df.replace({np.nan: None})
stations_df['DataStart'] = stations_df['DataStart'].apply(rearange_date_d_m_y)
stations_df['DataStop'] = stations_df['DataStop'].apply(rearange_date_d_m_y)
stations_df['Unit'] = stations_df[stations_df.columns[2]]
stations_df = stations_df.drop(stations_df.columns[2], axis=1)
stations_df['Tipologia'] = stations_df['Tipologia'].apply(map_sensor_type)
stations_df

,IdSensore,Tipologia,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,Unit
0,22006,Humidity,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,2019-10-14,None,N,5067376,591417,10.175407,45.753896,%
1,22007,Wind Direction,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,2019-10-14,None,N,5067376,591417,10.175407,45.753896,°
2,22003,Temperature,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,2019-10-14,None,N,5067376,591417,10.175407,45.753896,°C
3,22009,Wind Velocity,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,2019-10-14,None,N,5067376,591417,10.175407,45.753896,m/s
4,17612,Snow Depth,1545,San Siro Alpe Rescascìa,1285,CO,2019-11-07,None,N,5102069,518430,9.238323,46.071947,cm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,10787,Wind Velocity,1266,Bertonico v.Moro,65,LO,2022-04-12,None,N,5009105,552297,9.666250,45.233494,m/s
1244,19024,Global Radiation,1422,Bergamo v.Maffei,249,BG,2013-11-06,None,N,5060404,552526,9.674653,45.695197,W/m²
1245,7048,Wind Velocity,695,Ponti sul Mincio v.San Martino,113,MN,2006-12-19,2021-03-03,S,5030184,631718,10.683357,45.412772,m/s
1246,6990,Global Radiation,679,Valmadrera v.Pozzi,247,LC,2006-02-02,None,N,5076576,527307,9.351658,45.842207,W/m²


In [13]:
stations_df.columns

Index(['IdSensore', 'Tipologia', 'IdStazione', 'NomeStazione', 'Quota',
       'Provincia', 'DataStart', 'DataStop', 'Storico', 'UTM_Nord', 'UTM_Est',
       'lng', 'lat', 'Unit'],
      dtype='object')

In [15]:
base_url = "http://127.0.0.1:8000/api/collections/meteosensor"
body = {
    "bulk": True,
    "items": []
}

station_to_add = stations_df.copy()
for i, sensor in station_to_add.iterrows():
    item = {
        "sensor_id": sensor.IdSensore,
        "sensor_type": sensor.Tipologia,
        "measurement_unit": sensor.Unit,
        "station_id": sensor.IdStazione,
        "station_name": sensor.NomeStazione,
        "altitude": sensor.Quota,
        "province": sensor.Provincia,
        "is_historical": sensor.Storico == 'S',
        "date_start": sensor.DataStart, #YYYY-MM-DD
        "date_stop": sensor.DataStop, #YYYY-MM-DD
        "utm_north": sensor.UTM_Nord,
        "utm_east": sensor.UTM_Est,
        "latitude": sensor.lat,
        "longitude": sensor.lng,
        "location": f"POINT ({sensor.lng} {sensor.lat})"
    }
    body["items"].append(item)
    
body_json = json.dumps(body)

#uncomment for creating the stations
req = requests.post(base_url, body_json)
if req.status_code == 500:
    print(body_json)
print(f'request {i} - {req.status_code}')


request 1247 - 200


In [ ]:
stations